In [7]:
import pandas as pd

## Datasets & Cleaning

### Call of Duty

In [8]:
codbo3_data = pd.read_csv("../assets/raw/SteamDB Call of Duty Black Ops 3.csv")
codiw_data = pd.read_csv("../assets/raw/SteamDB Call of Duty Infinite Warfare.csv")
codwwii_data = pd.read_csv("../assets/raw/SteamDB Call of Duty WWII.csv")
codbo4_data = pd.read_csv("../assets/raw/SteamDB Call of Duty Black Ops 4.csv")
codmw_data = pd.read_csv("../assets/raw/SteamDB Call of Duty Modern Warfare.csv")
codbocw_data = pd.read_csv("../assets/raw/SteamDB Call of Duty Black Ops Cold War.csv")
codv_data = pd.read_csv("../assets/raw/SteamDB Call of Duty Vanguard.csv")
codlast3games_data = pd.read_csv("../assets/raw/SteamDB Call of Duty last3games.csv")

In [9]:
fields_to_keep = ['month', 'peak', 'gain', '% gain']
list_of_cod_data = [codbo3_data, codiw_data, codwwii_data, codbo4_data, codmw_data, codbocw_data, codv_data, codlast3games_data]

for i in range(len(list_of_cod_data)):
    df = list_of_cod_data[i]
    df.columns = df.columns.str.lower()
    df = df.replace("-", "0")
    df[['peak', 'gain', '% gain']] = df[['peak', 'gain', '% gain']].replace({",": "", "%": ""}, regex=True).astype(float)
    df = df[fields_to_keep][1:] # Getting rid of first row of Last 30 days
    df["month"] = pd.to_datetime(df["month"], format="%b-%y")
    df = df.sort_values("month")
    list_of_cod_data[i] = df


In [10]:
cod_merged = pd.concat(list_of_cod_data, ignore_index=True)
cod_merged = cod_merged[fields_to_keep]
cod_merged.head()

,month,peak,gain,% gain
0,2015-04-01,4.0,0.0,0.0
1,2015-05-01,12.0,8.0,200.0
2,2015-06-01,12.0,0.0,0.0
3,2015-07-01,18.0,6.0,50.0
4,2015-08-01,36.0,18.0,100.0


In [11]:
SteamDB_cod_merged_grouped = cod_merged.groupby(cod_merged.columns[0]).sum()

SteamDB_cod_merged_grouped = SteamDB_cod_merged_grouped.reset_index()
SteamDB_cod_merged_grouped = SteamDB_cod_merged_grouped.sort_values("month")

# Recalculate % gain
SteamDB_cod_merged_grouped['% gain'] = ((SteamDB_cod_merged_grouped['peak']-SteamDB_cod_merged_grouped['peak'].shift(1))*100/SteamDB_cod_merged_grouped['peak'].shift(1)).round(1)
SteamDB_cod_merged_grouped.iloc[0,3] = 0 # Setting the first % gain to 0

SteamDB_cod_merged_grouped = SteamDB_cod_merged_grouped[SteamDB_cod_merged_grouped["month"] >= "2020-06-01"] # Filtering rows before June 01 2020
SteamDB_cod_merged_grouped

,month,peak,gain,% gain
62,2020-06-01,8382.0,-3681.0,-22.7
63,2020-07-01,7746.0,-636.0,-7.6
64,2020-08-01,8607.0,861.0,11.1
65,2020-09-01,8060.0,-547.0,-6.4
66,2020-10-01,13277.0,5217.0,64.7
...,...,...,...,...
120,2025-04-01,152660.0,61207.0,66.9
121,2025-05-01,128330.0,-24330.0,-15.9
122,2025-06-01,106348.0,-21982.0,-17.1
123,2025-07-01,105203.0,-1145.0,-1.1


In [12]:
SteamDB_cod_merged_grouped.to_csv("../assets/clean/SteamDB_Call_of_Duty_Clean.csv",encoding="utf-8", index=False, header=True)